## Visualization utilities

In [3]:
#@markdown We implemented some functions to visualize the face landmark detection results. <br/> Run the following cell to activate the functions.

from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np
import matplotlib.pyplot as plt


def draw_landmarks_on_image(rgb_image, detection_result):
  face_landmarks_list = detection_result.face_landmarks
  annotated_image = np.copy(rgb_image)

  # Loop through the detected faces to visualize.
  for idx in range(len(face_landmarks_list)):
    face_landmarks = face_landmarks_list[idx]

    # Draw the face landmarks.
    face_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    face_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in face_landmarks
    ])

    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=mp.solutions.face_mesh.FACEMESH_TESSELATION,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp.solutions.drawing_styles
        .get_default_face_mesh_tesselation_style())
    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=mp.solutions.face_mesh.FACEMESH_CONTOURS,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp.solutions.drawing_styles
        .get_default_face_mesh_contours_style())
    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=mp.solutions.face_mesh.FACEMESH_IRISES,
          landmark_drawing_spec=None,
          connection_drawing_spec=mp.solutions.drawing_styles
          .get_default_face_mesh_iris_connections_style())

  return annotated_image

def plot_face_blendshapes_bar_graph(face_blendshapes):
  # Extract the face blendshapes category names and scores.
  face_blendshapes_names = [face_blendshapes_category.category_name for face_blendshapes_category in face_blendshapes]
  face_blendshapes_scores = [face_blendshapes_category.score for face_blendshapes_category in face_blendshapes]
  # The blendshapes are ordered in decreasing score value.
  face_blendshapes_ranks = range(len(face_blendshapes_names))

  fig, ax = plt.subplots(figsize=(12, 12))
  bar = ax.barh(face_blendshapes_ranks, face_blendshapes_scores, label=[str(x) for x in face_blendshapes_ranks])
  ax.set_yticks(face_blendshapes_ranks, face_blendshapes_names)
  ax.invert_yaxis()

  # Label each bar with values
  for score, patch in zip(face_blendshapes_scores, bar.patches):
    plt.text(patch.get_x() + patch.get_width(), patch.get_y(), f"{score:.4f}", va="top")

  ax.set_xlabel('Score')
  ax.set_title("Face Blendshapes")
  plt.tight_layout()
  plt.show()

In [1]:

import cv2

cap = cv2.VideoCapture(0)

In [13]:
from mvm_pb2 import BlendShapes

def encodeBlendShapes(blendShapes):
    out = BlendShapes()
    for element in blendShapes :
        blendShape = out.blend_shapes.add()
        blendShape.index = element.index
        blendShape.score = element.score
        blendShape.category_name = element.category_name
    return out

## Running inference and visualizing the results

Here are the steps to run face landmark detection using MediaPipe.

Check out the [MediaPipe documentation](https://developers.google.com/mediapipe/solutions/vision/face_landmarker/python) to learn more about configuration options that this task supports.


In [14]:
# STEP 1: Import the necessary modules.
import mediapipe as mp

from mediapipe.tasks import python
from mediapipe.tasks.python import vision

# STEP 2: Create an FaceLandmarker object.
base_options = python.BaseOptions(model_asset_path='./model/face_landmarker.task')
options = vision.FaceLandmarkerOptions(base_options=base_options,
                                       output_face_blendshapes=True,
                                       output_facial_transformation_matrixes=True,
                                       num_faces=1)
detector = vision.FaceLandmarker.create_from_options(options)


while cap.isOpened():
    # STEP 3: Load the input image.
    success, image = cap.read()
    image = mp.Image(image_format=mp.ImageFormat.SRGB, data=image)

    # STEP 4: Detect face landmarks from the input image.
    detection_result = detector.detect(image)

    # STEP 5: Process the detection result. In this case, visualize it.
    annotated_image = draw_landmarks_on_image(image.numpy_view(), detection_result)
    # print(detection_result.face_blendshapes[0][2])
    print(encodeBlendShapes(detection_result.face_blendshapes[0]))


    

blend_shapes {
  score: 1.7351502492601867e-06
  category_name: "_neutral"
}
blend_shapes {
  index: 1
  score: 0.17917104065418243
  category_name: "browDownLeft"
}
blend_shapes {
  index: 2
  score: 0.14327219128608704
  category_name: "browDownRight"
}
blend_shapes {
  index: 3
  score: 0.02167472243309021
  category_name: "browInnerUp"
}
blend_shapes {
  index: 4
  score: 0.027108164504170418
  category_name: "browOuterUpLeft"
}
blend_shapes {
  index: 5
  score: 0.02936186082661152
  category_name: "browOuterUpRight"
}
blend_shapes {
  index: 6
  score: 1.7276690414291807e-05
  category_name: "cheekPuff"
}
blend_shapes {
  index: 7
  score: 7.73895010297565e-07
  category_name: "cheekSquintLeft"
}
blend_shapes {
  index: 8
  score: 1.4412105997507751e-07
  category_name: "cheekSquintRight"
}
blend_shapes {
  index: 9
  score: 0.03469244763255119
  category_name: "eyeBlinkLeft"
}
blend_shapes {
  index: 10
  score: 0.015445523895323277
  category_name: "eyeBlinkRight"
}
blend_shape

KeyboardInterrupt: 

We will also visualize the face blendshapes categories using a bar graph.

In [4]:
plot_face_blendshapes_bar_graph(detection_result.face_blendshapes[0])

IndexError: list index out of range

And print the transformation matrix.

In [ ]:
print(detection_result.facial_transformation_matrixes)